# Recovering a 2d function from its gradient with a Gaussian Process Regression model

This notebook shows examples on how to use the present gaussian process regression framework to formally integrate functions from derivative observations.

This example will show how to predict a 2D function using gradient observations with both a full GPR and a sparse GPR. The sparse GPR framework works by projecting the training data into a lower dimensional feature space in order to reduce the computational cost (mainly matrix inversions).

## Creating the training data set

`jax.numpy` has almost the same usage as the standard `numpy` package, with the caveat that `jax.ndarray` is an immutable type, meaning that no inplace changes can be made. For creating training data this should however not be an issue.

In [ ]:
import jax.numpy as jnp
from jax import random

We will model a forth order polynomial:

In [ ]:
def fun(x, noise=0.0, key = random.PRNGKey(0)):
    return (x[:,0]**2 + x[:,1] - 11)**2 / 800.0 + (x[:,0] + x[:,1]**2 -7)**2 / 800.0 + random.normal(key,(len(x),), dtype=jnp.float32)*noise

def grad(x, noise=0.0, key = random.PRNGKey(0)):
    dx1 = 4 * (x[:,0]**2 + x[:,1] - 11) * x[:,0] + 2 * (x[:,0] + x[:,1]**2 -7)
    dx2 = 2 * (x[:,0]**2 + x[:,1] - 11) + 4 * (x[:,0] + x[:,1]**2 -7) * x[:,1]

    return jnp.vstack((dx1, dx2)).T / 800.0 + random.normal(key,x.shape, dtype=jnp.float32)*noise

To define the training data we first need to define boundaries to choose the datapoints from. Then, random points are chosen in this interval. `random.split` creates a new subkey from the previous key to get a new sudo-random sample from the uniform distribution.

In [ ]:
# Interval bounds from which to choose the data points
bounds = jnp.array([-5.0, 5.0])

# How many function and derivative observations should be chosen
num_f_vals = 1
num_d_vals = 100

# initial seed for the pseudo random key generation
seed = 3

# create new keys and randomly sample the above interval for training features
key, subkey = random.split(random.PRNGKey(seed))
x_func = random.uniform(subkey, (num_f_vals, 2), minval=bounds[0], maxval=bounds[1])
key, subkey = random.split(key)
x_der = random.uniform(subkey, (num_d_vals,2), minval=bounds[0], maxval=bounds[1])

# noise with which to sample the training labels
noise = 0.02
key, subkey = random.split(key)
y_func = fun(x_func,noise, subkey)
key, subkey = random.split(key)
y_der = grad(x_der, noise, subkey)

The GPR framework needs as input for training a tuple of arrays `X_split` of which contains a set of points where the function is sampled and a set of points where the gradient is sampled. Both array in `X_split` is of shape `(n_samples_i, N)`. `X_split` should be ordered as follows: the first array represents the datapoints for the function observations and the second array represents the gradient of the function. `Y_train` should just be an array of shape `(n_samples_function + n_samples_gradient,)`.

In [ ]:
X_split = [x_func,x_der]

Y_train = jnp.hstack((y_func, y_der.reshape(-1)))

### Defining the Kernel and its initial parameters

The kernels can be found in `jaxgp.kernels`. Currently implemented are `RBF`, `Linear`, and `Periodic` kernels. When in doubt what kernel to use, go with an `RBF` kernel.

In [ ]:
from jaxgp.kernels import RBF

kernel = RBF()
# an RBF kernel has per default 2 parameters
init_kernel_params = jnp.ones(2)*jnp.log(2)

## The full GPR model

### Training the full GPR model

The regression models can be found in `jaxgp.regression`. The `ExactGPR` model takes as inital arguments a kernel to use, initial parameters for the kernel and a noise parameter that underlies the noisy datageneration.

Additionally one can also give the model a logging function that saves the parameters at each step of the optimization. For this a convenience class `Logger` can be found in `jaxgp.utils`.

In [ ]:
from jaxgp.regression import ExactGPR
from jaxgp.utils import Logger

# logger = Logger()
model = ExactGPR(kernel, init_kernel_params, noise, optimize_noise=True) #, logger=logger)

In [ ]:
model.train(X_split, Y_train)

Iterations of the parameters during optimization.

In [ ]:
# print(logger.iters_list)

### Making predictions with the trained model

We will make predictions on an evenly spaced grid of $101\times101$ points square box defined by the above bounds.

In [ ]:
predict_grid = jnp.array(jnp.meshgrid(jnp.linspace(*bounds, 101), jnp.linspace(*bounds, 101))).T.reshape(101**2, 2)

# model.eval returns a mean prediction and a confidence interval around the mean prediction
means, stds = model.eval(predict_grid)

### Visualizing the result

Since the system is now 2D we squared the number of function and gradient observations. Comparing the contour plot of the prediction with the true function, the results are quite passable. However, in the 2D case it looks like there is a need for more function evaluations. Maybe also at some strategic points.

In [ ]:
import matplotlib.pyplot as plt

means = means.reshape(101,101)
stds = stds.reshape(101,101)
true = fun(predict_grid).reshape(101,101)

fig, ax = plt.subplots(2, 2, figsize=(10,7))

mesh = jnp.meshgrid(jnp.linspace(*bounds, 101),jnp.linspace(*bounds, 101))

im1 = ax[0,0].contourf(*mesh, means, levels=12, vmin=-0.15, vmax=1.2)
im2 = ax[0,1].contourf(*mesh, true, levels=12, vmin=-0.15, vmax=1.2)
im3 = ax[1,0].contourf(*mesh, stds)
im4 = ax[1,1].contourf(*mesh, jnp.abs(means-true))
# im4 = ax[1,1].contourf(*mesh, jnp.greater(true, means-stds)*jnp.less(true, means+stds))

ax[1,1].scatter(x_der[:,0], x_der[:,1], c="r", marker="x", label="der pos")
ax[1,1].scatter(x_func[:,0], x_func[:,1], c="orange", marker="+", label="fun pos")

ax[1,0].scatter(x_der[:,0], x_der[:,1], c="r", marker="x", label="der pos")
ax[1,0].scatter(x_func[:,0], x_func[:,1], c="orange", marker="+", label="fun pos")

plt.colorbar(im1, ax=ax[0,0])
plt.colorbar(im2, ax=ax[0,1])
plt.colorbar(im3, ax=ax[1,0])
plt.colorbar(im4, ax=ax[1,1])

ax[0,0].set_title("prediction")
ax[0,1].set_title("true function")
ax[1,0].set_title("std")
ax[1,1].set_title("abs dif")

## The sparse GPR model

### Training the sparse GPR model

The `sparseGPR` model can be found in `jaxgp.regression`. The idea of a sparse model is to project your training data into a space with smaller dimension in order to save in computational cost. This is done by projecting your full training set onto a set of reference points via the kernel. 

There are typically 3 methods to choose reference points:
 - choosing a subset of size $m<n$ from the existing datapoints
 - creating an even grid inside the bounds on which the model should be evaluated
 - Choosing a fixed number of reference points and then optimizing their locations

Note that below the evenly spread reference grid has fewer points than the subset grid. This was done in order to get a nice even grid which was achieved by taking the largerst perfect square smaller than the number of reference points.

Furthermore we also created a larger set of gradient observations since the predictive power of the sparse model is lower than of the full model. However, this is not a problem computation wise as is seen further below.

In [ ]:
from jaxgp.regression import SparseGPR

# How many function and derivative observations should be chosen
num_f_vals = 1
num_d_vals = 1000

# initial seed for the pseudo random key generation
seed = 0

# create new keys and randomly sample the above interval for training features
key, subkey = random.split(random.PRNGKey(seed))
x_func = random.uniform(subkey, (num_f_vals, 2), minval=bounds[0], maxval=bounds[1])
key, subkey = random.split(key)
x_der = random.uniform(subkey, (num_d_vals,2), minval=bounds[0], maxval=bounds[1])

# noise with which to sample the training labels
noise = 0.02
key, subkey = random.split(key)
y_func = fun(x_func,noise, subkey)
key, subkey = random.split(key)
y_der = grad(x_der, noise, subkey)

logger = Logger()

num_ref_points = (num_d_vals + num_f_vals) // 10
key, subkey = random.split(key)
X_ref_rand = random.permutation(subkey, jnp.vstack((x_der,x_func)))[:num_ref_points]
# this grid has fewer points. It has N points, where N is the largest perfect square smaller than num_ref_points
X_ref_even = jnp.array(jnp.meshgrid(jnp.linspace(*bounds, round(jnp.sqrt(num_ref_points))),jnp.linspace(*bounds, round(jnp.sqrt(num_ref_points))))).reshape(2,-1).T

In [ ]:
optimize_noise = True
optimize_ref = True

model_rand = SparseGPR(kernel, init_kernel_params, noise, X_ref_rand, logger=None)
model_even = SparseGPR(kernel, init_kernel_params, noise, X_ref_even, logger=None)
model_optim = SparseGPR(kernel, init_kernel_params, noise, X_ref_rand, optimize_noise=optimize_noise, optimize_ref=optimize_ref, logger=None)

In [ ]:
X_split = [x_func,x_der]

Y_train = jnp.hstack((y_func, y_der.reshape(-1)))

Looking at the time needed to train the sparse models, even with 10 times as many datapoints the computation is still twice as fast, compare to the full model. This is because the computational effort is roughly 100 times smaller: 
 - The full model needs $\mathcal{O}(N^3)$ flops to train and fit the model
 - The sparse model needs $\mathcal{O}(M^2N + M^3)$ flops to train and fit the model

which comes out to roughly a factor 100 times faster.

In [ ]:
model_rand.train(X_split, Y_train)
model_even.train(X_split, Y_train)
model_optim.train(X_split, Y_train)

### Making predictions with the trained model

We will make predictions on an evenly spaced grid of $101\times101$ points square box defined by the above bounds.

In [ ]:
predict_grid = jnp.array(jnp.meshgrid(jnp.linspace(*bounds, 101), jnp.linspace(*bounds, 101))).T.reshape(101**2, 2)

# model.eval returns a mean prediction and a confidence interval around the mean prediction
means_rand, stds_rand = model_rand.eval(predict_grid)
means_even, stds_even = model_even.eval(predict_grid)
means_optim, stds_optim = model_optim.eval(predict_grid)

### Visualizing the result

#### Random subset reference points

In [ ]:
import matplotlib.pyplot as plt

means = means_rand.reshape(101,101)
stds = stds_rand.reshape(101,101)
true = fun(predict_grid).reshape(101,101)

fig, ax = plt.subplots(2, 2, figsize=(10,7))

mesh = jnp.meshgrid(jnp.linspace(*bounds, 101),jnp.linspace(*bounds, 101))

im1 = ax[0,0].contourf(*mesh, means, levels=12)#, vmin=-0.15, vmax=1.2)
im2 = ax[0,1].contourf(*mesh, true, levels=12)#, vmin=-0.15, vmax=1.2)
im3 = ax[1,0].contourf(*mesh, stds)
im4 = ax[1,1].contourf(*mesh, jnp.abs(means-true))

plt.colorbar(im1, ax=ax[0,0])
plt.colorbar(im2, ax=ax[0,1])
plt.colorbar(im3, ax=ax[1,0])
plt.colorbar(im4, ax=ax[1,1])

ax[1,1].scatter(X_ref_rand[:,0], X_ref_rand[:,1], c="r", marker="x", label="der pos")

ax[1,0].scatter(X_ref_rand[:,0], X_ref_rand[:,1], c="r", marker="x", label="der pos")

ax[0,0].set_title("prediction")
ax[0,1].set_title("true function")
ax[1,0].set_title("std")
ax[1,1].set_title("abs dif")

#### Evenly spaced reference points

Note that both produce similar results in this case. However, the evenly spaced grid lacks some stability and for different configurations the resulting standard deviations can't be calculated

In [ ]:
means = means_even.reshape(101,101)
stds = stds_even.reshape(101,101)
true = fun(predict_grid).reshape(101,101)

fig, ax = plt.subplots(2, 2, figsize=(10,7))

mesh = jnp.meshgrid(jnp.linspace(*bounds, 101),jnp.linspace(*bounds, 101))

im1 = ax[0,0].contourf(*mesh, means, levels=12, vmin=-0.15, vmax=1.2)
im2 = ax[0,1].contourf(*mesh, true, levels=12, vmin=-0.15, vmax=1.2)
im3 = ax[1,0].contourf(*mesh, stds)
im4 = ax[1,1].contourf(*mesh, jnp.abs(means-true))

plt.colorbar(im1, ax=ax[0,0])
plt.colorbar(im2, ax=ax[0,1])
plt.colorbar(im3, ax=ax[1,0])
plt.colorbar(im4, ax=ax[1,1])

ax[1,1].scatter(X_ref_even[:,0], X_ref_even[:,1], c="r", marker="x", label="der pos")

ax[1,0].scatter(X_ref_even[:,0], X_ref_even[:,1], c="r", marker="x", label="der pos")

ax[0,0].set_title("prediction")
ax[0,1].set_title("true function")
ax[1,0].set_title("std")
ax[1,1].set_title("abs dif")

#### Optimized grid

Note that some reference points are located outside of the region where the function was defined. However, this should not be a problem.

In [ ]:
means = means_optim.reshape(101,101)
stds = stds_optim.reshape(101,101)
true = fun(predict_grid).reshape(101,101)

fig, ax = plt.subplots(2, 2, figsize=(10,7))

mesh = jnp.meshgrid(jnp.linspace(*bounds, 101),jnp.linspace(*bounds, 101))

im1 = ax[0,0].contourf(*mesh, means, levels=12, vmin=-0.15, vmax=1.2)
im2 = ax[0,1].contourf(*mesh, true, levels=12, vmin=-0.15, vmax=1.2)
im3 = ax[1,0].contourf(*mesh, stds)
im4 = ax[1,1].contourf(*mesh, jnp.abs(means-true))

plt.colorbar(im1, ax=ax[0,0])
plt.colorbar(im2, ax=ax[0,1])
plt.colorbar(im3, ax=ax[1,0])
plt.colorbar(im4, ax=ax[1,1])

ax[1,1].scatter(model_optim.X_ref[:,0], model_optim.X_ref[:,1], c="r", marker="x", label="der pos")

ax[1,0].scatter(model_optim.X_ref[:,0], model_optim.X_ref[:,1], c="r", marker="x", label="der pos")

ax[0,0].set_title("prediction")
ax[0,1].set_title("true function")
ax[1,0].set_title("std")
ax[1,1].set_title("abs dif")